In [1]:
import pandas as pd
import json
import ast
import seaborn as sns
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats

In [2]:
releases = pd.read_csv('../Data_input/releases.csv')
tender = pd.read_csv('../Data_input/tender.csv')
awards = pd.read_csv('../Data_input/awards.csv')
parties = pd.read_csv('../Data_input/parties.csv')
montos = pd.read_csv('../Data_input/Montos.csv')
items = pd.read_csv('../Data_input/tender_items.csv')

## Preprocesado Tender

In [3]:
tender['tp_endDate'] = pd.to_datetime(tender['tp_endDate']).dt.date
tender['tp_endDate'] = pd.to_datetime(tender['tp_endDate'])

In [4]:
tender['tp_endDate'] = pd.to_datetime(tender['tp_endDate'])

## Preprocesado Awards

In [5]:
awards['date'] = pd.to_datetime(awards['date']).dt.date
awards['date'] = pd.to_datetime(awards['date'])

## Preprocesado Releases

In [6]:
releases['contracts_dateSigned'] = pd.to_datetime(releases['contracts_dateSigned']).dt.date
releases['contracts_dateSigned'] = pd.to_datetime(releases['contracts_dateSigned'])

## Red Flags

In [7]:
def NF003(df): #tender
    limited=df.loc[(df["procurementMethod"]=="limited")&(df["tp_durationInDays"]<10)]["ocid"].tolist()
    direct=df.loc[(df["procurementMethod"]=="direct")&(df["tp_durationInDays"]<9)]["ocid"].tolist() 
    selective = df.loc[(df["procurementMethod"]=="selective")&(df["tp_durationInDays"]<15)]["ocid"].tolist()
    mopen=df.loc[(df["procurementMethod"]=="open")&(df["tp_durationInDays"]<15)]["ocid"].tolist()
    redflag_NF003={"open":mopen,"selective":selective,"limited":limited,"direct":direct}
    df_NF003=pd.DataFrame.from_dict(redflag_NF003,orient="index").transpose().stack().reset_index()
    df_NF003.columns=["Nivel","procurementMethod","ocid"]
    df_NF003=df_NF003.drop(columns="Nivel")
    return df_NF003

In [8]:
def NF010(df,df_montos): #tender, montos
    df_montos_bienes=df_montos.loc[df_montos["Tipo"]=="Bienes"]#Separar los umbrales a utilizar
    df_montos_obras=df_montos.loc[df_montos["Tipo"]=="Obras"]
    #bienes
    df_value_bienes_norm = df.loc[df["mainProcurementCategory"].isnull()]
    df_value_bienes=df.loc[(df["mainProcurementCategory"]=="goods")]
    df_value_bienes=df_value_bienes.merge(df_montos_bienes,on ="procurementMethodDetails",how ="left")
    #Se verifica que el valor de amount se encuentre dentro de los límites, para este redflag se requiere
    #saber, si los valores se encuentran justo por debajo del límite, es decir +-10% del valor establecido
    bienes=df_value_bienes.loc[(df_value_bienes["value_amount"]>=(df_value_bienes["menor"]*0.9))|(df_value_bienes["value_amount"]<(df_value_bienes["mayor"]*0.9))]["ocid"].tolist()
    #Obras
    df_value_obras =df.loc[(df["mainProcurementCategory"]=="works")]
    df_value_obras=df_value_obras.merge(df_montos_obras,on = "procurementMethodDetails",how = "left")
    obras=df_value_obras.loc[(df_value_obras["value_amount"]>=(df_value_obras["menor"]*0.9))|(df_value_obras["value_amount"]<(df_value_obras["mayor"]*0.9))]["ocid"].tolist()
    #Juntar los Ocid
    redflag_NF010={"bienes":bienes,"obras":obras}
    df_NF010=pd.DataFrame.from_dict(redflag_NF010,orient="index").transpose().stack().reset_index()
    df_NF010.columns=["Nivel","mainProcurementCategory","ocid"]
    df_NF010=df_NF010.drop(columns="Nivel")
    return df_NF010

In [9]:
def NF011():
    df_items=items.merge(tender[["ocid","tp_startDate","procuringEntity_name","procurementMethod","procurementMethodDetails"]],on="ocid",how ="left")
    df_items["tp_startDate"]=pd.to_datetime(df_items["tp_startDate"])
    #Se realiza una agrupación por método, entidad contratante y tipo de compra, sacando un total del valor adjunto a cada grupo
    dfg_items=df_items.groupby(["procuringEntity_name","classification_id","procurementMethod","procurementMethodDetails",pd.Grouper(key="tp_startDate",freq="3M")])["unit_value_amount"].sum().reset_index()
    #Se separa cada una de las distintas secciones a comparar con los datos externos
    dfg_competitive = dfg_items.loc[(dfg_items["procurementMethod"]!="direct")]
    dfg_non_competitve = dfg_items.loc[(dfg_items["procurementMethod"]=="direct")]
    threshold = montos.loc[(montos["Tipo"]!="Obras")]
    dfg_competitive=dfg_competitive.merge(threshold[["procurementMethodDetails","mayor","menor"]], on="procurementMethodDetails", how="left")
    dfg_non_competitve=dfg_non_competitve.merge(threshold[["procurementMethodDetails","mayor","menor"]], on="procurementMethodDetails", how="left")
    entity_competitive = dfg_competitive.loc[(dfg_competitive["unit_value_amount"]>=(dfg_competitive["menor"]))|(dfg_competitive["unit_value_amount"]<(dfg_competitive["mayor"]))]["procuringEntity_name"].tolist()
    entity_non_competitive = dfg_non_competitve.loc[(dfg_non_competitve["unit_value_amount"]>=(dfg_non_competitve["menor"]))|(dfg_non_competitve["unit_value_amount"]<(dfg_non_competitve["mayor"]))]["procuringEntity_name"].tolist()
    redflag_NF011={"Competitive":entity_competitive,"Non_competitive":entity_non_competitive}
    df_NF011=pd.DataFrame.from_dict(redflag_NF011,orient="index").transpose().stack().reset_index()
    df_NF011.columns=["Nivel","procurementMethod","procuringEntity_name"]
    df_NF011=df_NF011.drop(columns=["Nivel","procurementMethod"])
    df_NF011.drop_duplicates(inplace=True)
    df_NF011.reset_index(drop=True, inplace=True)
    return df_NF011

In [ ]:
NF011()

In [10]:
def NF016():
    df_items=items.merge(tender[["ocid","value_amount"]],on="ocid",how ="left")
    dfg_items=df_items.groupby(['ocid','classification_id'])["unit_value_amount"].mean().reset_index()
    ocid_16=dfg_items.loc[(stats.zscore(dfg_items["unit_value_amount"])>3)|(stats.zscore(dfg_items["unit_value_amount"])<-3)]["ocid"].tolist()
    df_NF016=pd.DataFrame(ocid_16)
    df_NF016.columns=["ocid"]
    return df_NF016

In [11]:
def NF018():
    ocid_18=tender.loc[(tender['procurementMethod']!="direct")&(tender['numberOfTenderers']<=1)]["ocid"].tolist()
    df_NF018=pd.DataFrame(ocid_18)
    df_NF018.columns=["ocid"]
    return df_NF018

In [12]:
def NF019():
    df_items=items.merge(tender[["ocid","procuringEntity_name","procurementMethod","tp_startDate","numberOfTenderers"]],on="ocid",how ="left")
    df_items["tp_startDate"]=pd.to_datetime(df_items["tp_startDate"])
    dfg_items=df_items.groupby(["ocid",'classification_id',"procuringEntity_name","procurementMethod",pd.Grouper(key="tp_startDate",freq="M")])["numberOfTenderers"].sum().reset_index()
    mean_byitem = df_items.groupby(['classification_id',pd.Grouper(key="tp_startDate",freq="M")])["numberOfTenderers"].sum().reset_index()
    mean_byitem.columns = ['classification_id', 'tp_startDate', 'mean_numberOfTenderers']
    dfg_items=dfg_items.merge(mean_byitem[['classification_id','mean_numberOfTenderers']],on='classification_id',how ="left")
    entity_19 = dfg_items.loc[dfg_items["numberOfTenderers"]>dfg_items['mean_numberOfTenderers']]["procuringEntity_name"].tolist()
    df_NF019=pd.DataFrame(entity_19)
    df_NF019.columns=["procuringEntity_name"]
    df_NF019.drop_duplicates(inplace=True)
    df_NF019.reset_index(drop=True, inplace=True)
    
    return df_NF019

In [13]:
def NF025():
    # Solo participación de tenderers
    tenderers_id = pd.DataFrame(parties[parties['roles'] != 'buyer'].groupby('id')['roles'].value_counts())
    tenderers_id.rename(columns={'roles':'participations'}, inplace=True)
    tenderers_id.reset_index(inplace=True)
    tenderers_id = tenderers_id[tenderers_id['roles'] == 'tenderer']
    
    # Ganadores de cada ocid
    winners = awards['supplier_id'].value_counts().reset_index()
    winners.rename(columns = {'index':'id', 'supplier_id':'wins'}, inplace=True)
    
    # Ganadores y participaciones
    participations = winners.merge(tenderers_id[['id','participations']], on='id', how='outer')
    participations['wins'].fillna(0, inplace=True)
    participations['participations'].fillna(0, inplace=True)
    participations['participations'] = participations['wins'] + participations['participations']

    # Límite de ids dependiendo de la tasa de éxitos
    participations['success_rate'] = participations['wins']/participations['participations']
    red_flag = participations[(participations['success_rate'] < 0.75) & (participations['participations'] > 3)]
    red_flag = red_flag.sort_values(by='participations', ascending=False)
    red_flag.reset_index(drop=True, inplace=True)
    
    # Se retorna una lista de DataFrames - Índice 0 participaciones - Índice 1 Ids que generan red-flag
    return [participations, red_flag]

In [14]:
def NF026():
    # Limitando la obtención de Suppliers y Tenderers
    supp_tend = parties[parties['roles'] != 'buyer']
    
    # Con ayuda del red-flag NF025 se obtienen las participaciones para limitar los ids necesarios
    participations = NF025()[0]
    participations = participations[(participations['success_rate'] < 1) & (participations['wins'] > 1)]
    ocids_supp_tend = supp_tend[supp_tend['id'].isin(participations['id'])]['ocid']
    supp_tend = supp_tend[supp_tend['ocid'].isin(ocids_supp_tend)]
    
    # Creación de tabla separando los los suppliers y tenderers para cada OCID 
    tb = pd.pivot_table(supp_tend, values='id', index='ocid', columns='roles', aggfunc=list)
    suppliers = tb['supplier'].values.tolist()
    suppliers = [x[0] for x in suppliers]
    tb['supplier'] = suppliers
    
    # Diccionario de conteo de aparición de un participante en presencia de un ganador
    dic = {}
    suppliers = tb['supplier'].values.tolist()
    tenderers = tb['tenderer'].values.tolist()
    for supp, tend in zip(suppliers,tenderers):
        if supp not in dic:
            if type(tend) is list:
                dic[supp] = {i:1 for i in tend}
        else:
            if type(tend) is list:
                for i in tend:
                    if i not in dic[supp]:
                        dic[supp][i] = 1
                    else:
                        dic[supp][i] += 1
    
    dicDF = pd.DataFrame(dic)
    dicDF.sort_index(axis=0, inplace=True)
    dicDF.sort_index(axis=1, inplace=True)
    dicDF.fillna(0, inplace=True)
    
    participants = dicDF.where(dicDF > 1).dropna(how='all').index.values.tolist()
    suppliers_winners = dicDF.where(dicDF > 1).dropna(how='all',axis=1).columns.values.tolist()
    total = participants + suppliers_winners
    red_flag = pd.DataFrame(parties[parties['id'].isin(total)].groupby(['ocid','id'])['roles'].sum()).reset_index()
    
    return [dicDF, red_flag]

In [15]:
def region(x):
    lst = x['roles']
    addr = x['address_region']
    ids = x['id']
    regi = ''
    for i,l in enumerate(lst):
        if 'buyer' in l:
            regi = addr[i]
            ids.pop(i) 
    return [ids,regi,len(ids)]

In [16]:
def NF027():
    parties_region = parties[['ocid','roles','id','address_region']]
    realeses_buyer = releases[['ocid','buyer_id','buyer_name']]
    df = realeses_buyer.merge(parties_region, on='ocid')
    
    roles_regions = df.groupby('ocid').agg({'roles':list,'id':list,'address_region':list})
    roles_regions[['id_ofertantes','provincia','total_ofertas']] = roles_regions.apply(region, axis=1, result_type='expand')
    roles_regions.drop(columns=['roles','address_region','id'], inplace=True)
    roles_regions = roles_regions[roles_regions['total_ofertas'] > 5]
    
    return roles_regions.reset_index()

In [17]:
def NF031():
    reles = releases[['ocid','buyer_id','buyer_name','planning_budget_amount']]
    aw =awards[['ocid','amount','id_awards']]
    df = reles.merge(aw, on='ocid')
    df.rename(columns={'id_awards':'supplier_id'}, inplace=True)
    df['amount/budget'] = df['amount'] / df['planning_budget_amount']
    df.fillna(0, inplace=True)
    red_flag = df[df['amount/budget'] > 1]
    
    return red_flag

In [18]:
def NF042(df): #parties
    df=df.loc[df["roles"]!="buyer"]
    df_telefono=df.loc[df["contactPoint_telephone"].notnull()]
    df_telefono["contactPoint_telephone"] = df_telefono["contactPoint_telephone"].astype(str)
    df_telefono=df_telefono.loc[df_telefono["contactPoint_telephone"].str.len()!=10]
    df_postal=df.loc[df["address_postalCode"].notnull()]
    df_postal["address_postalCode"] = df_postal["address_postalCode"].astype(str)
    df_postal=df_postal.loc[df_postal["address_postalCode"].str.len()!=6]
    df_direccion=df.loc[df["address_streetAddress"].isnull()]
    df_final=pd.concat([df_telefono,df_postal,df_direccion])
    df_final.drop_duplicates(subset=["ocid"], keep="last", inplace = True)
    lista_ocid=list(df_final["ocid"])
    dicc={"ocid":lista_ocid}
    df_retorno=pd.DataFrame(dicc)
    return df_retorno

In [19]:
def NF043(df): #parties
    df_telefono=df.loc[df["contactPoint_telephone"].notnull()]
    df_telefono=df_telefono.loc[df_telefono.duplicated(subset=["ocid","contactPoint_telephone"],keep=False)]
    df_telefono=df_telefono.groupby("ocid")["roles"].sum().reset_index()
    f1,c1=df_telefono.shape
    if f1!=0:
        df_telefono=df_telefono.loc[df_telefono["roles"].str.contains("buyer")]
    df_postal=df.loc[df["address_postalCode"].notnull()]
    df_postal=df_postal.loc[df_postal.duplicated(subset=["ocid","address_postalCode"],keep=False)]
    df_postal=df_postal.groupby("ocid")["roles"].sum().reset_index()
    f2,c2=df_postal.shape
    if f2!=0:
        df_postal=df_postal.loc[df_postal["roles"].str.contains("buyer")]
    df_direccion=df.loc[df["address_streetAddress"].notnull()]
    df_direccion=df_direccion.loc[df_direccion.duplicated(subset=["ocid","address_streetAddress"],keep=False)]
    df_direccion=df_direccion.groupby("ocid")["roles"].sum().reset_index()
    f3,c3=df_direccion.shape
    if f3!=0:
        df_direccion=df_direccion.loc[df_direccion["roles"].str.contains("buyer")]
    df_final=pd.concat([df_telefono,df_postal,df_direccion])
    df_final.drop_duplicates(subset=["ocid"], keep="last", inplace = True)
    lista_ocid=list(df_final["ocid"])
    dicc={"ocid":lista_ocid}
    df_retorno=pd.DataFrame(dicc)
    return df_retorno

In [20]:
def NF044(df): #parties
    df=df.loc[df["roles"]!="buyer"]
    df_telefono=df.loc[df["contactPoint_telephone"].notnull()]
    df_telefono=df_telefono.loc[df_telefono.duplicated(subset=["ocid","contactPoint_telephone"],keep=False)]
    df_postal=df.loc[df["address_postalCode"].notnull()]
    df_postal=df_postal.loc[df_postal.duplicated(subset=["ocid","address_postalCode"],keep=False)]
    df_direccion=df.loc[df["address_streetAddress"].notnull()]
    df_direccion=df_direccion.loc[df_direccion.duplicated(subset=["ocid","address_streetAddress"],keep=False)]
    df_name=df.loc[df["contactPoint_name"].notnull()]
    df_name=df_name.loc[df_name.duplicated(subset=["ocid","contactPoint_name"],keep=False)]
    df_email=df.loc[df["contactPoint_email"].notnull()]
    df_email=df_email.loc[df_email.duplicated(subset=["ocid","contactPoint_email"],keep=False)]
    df_final=pd.concat([df_telefono,df_postal,df_direccion,df_name,df_email])
    df_final.drop_duplicates(subset=["ocid"], keep="last", inplace = True)
    lista_ocid=list(df_final["ocid"])
    dicc={"ocid":lista_ocid}
    df_retorno=pd.DataFrame(dicc)
    return df_retorno

In [21]:
def NF047(df): #tender
    df=df.loc[df["roles"]=="supplier"]
    df_url=df.loc[df["contactPoint_url"].isnull()]
    lista_ocid=list(df_url["ocid"])
    dicc={"ocid":lista_ocid}
    df_retorno=pd.DataFrame(dicc)
    return df_retorno

In [22]:
def tenderPeriod():
    df_period = tender[['ocid', 'tp_endDate', 'procurementMethod']].merge(awards[['ocid', 'date']], on='ocid')
    df_period['decision_period'] = df_period['date'] - df_period['tp_endDate']
    df_period = df_period.loc[df_period['decision_period'].notnull()].reset_index(drop=True)
    df_period['decision_period'] = df_period['decision_period'].dt.days
    df_period = df_period[df_period['procurementMethod'] != 'direct']
    return df_period

In [23]:
def NF063():
    df_period = tenderPeriod()
    mascara = df_period['decision_period'] >= (df_period['decision_period'].mean() * 1.5)
    outliers = df_period.loc[mascara]
    return outliers

In [24]:
def NF062():
    df_period = tenderPeriod()
    mascara = df_period['decision_period'] <= (df_period['decision_period'].mean() * 0.5)
    outliers = df_period.loc[mascara]
    return outliers

In [25]:
def NF061():
    df_period = releases[['ocid', 'contracts_dateSigned']].merge(awards[['ocid', 'date']], on='ocid')
    df_period['contract_period'] = df_period['contracts_dateSigned'] - df_period['date']
    df_period = df_period.loc[df_period['contract_period'].notnull()].reset_index(drop=True)
    df_period['contract_period'] = df_period['contract_period'].dt.days
    # Outliners decision_period
    q3 = df_period['contract_period'].quantile(0.75)
    q1 = df_period['contract_period'].quantile(0.25)
    iqr = q3 - q1
    mascara = (df_period['contract_period'] > (q3 + 1.5*iqr))
    outliers = df_period.loc[mascara]
    return outliers

In [26]:
def NF059():
    df_amount = awards[['ocid', 'amount']].merge(releases[['ocid', 'contracts_value_amount']], on='ocid')
    df_amount['amount_difference'] = df_amount['contracts_value_amount'] - df_amount['amount']
    df_amount = df_amount[(df_amount['amount_difference'].notnull()) & (df_amount['amount_difference'] != 0)]
    df_amount['difference_perc'] = abs(df_amount['amount_difference'] / df_amount['amount'])
    df_amount = df_amount[df_amount['difference_perc'] > 0.3]
    return df_amount

In [27]:
df1 = awards[['ocid', 'amount', 'supplier_name']]
df2 = tender[['ocid', 'procurementMethod', 'mainProcurementCategory', 'procurementMethodDetails']]
dfsup = df1.merge(df2, on='ocid')
dfsup = dfsup[dfsup['procurementMethod'] == 'direct']

In [28]:
dfg = dfsup.groupby('supplier_name').agg({'amount':[np.sum, np.size]})
dfg.columns = dfg.columns.droplevel()
dfg = dfg.reset_index()

In [29]:
def NF055():
    dfsup = awards[['ocid', 'supplier_id', 'supplier_name']].merge(tender[['ocid', 'procurementMethod']], on='ocid')

In [ ]:
NF059()

## Almacenamiento de red flags

In [30]:
def generate_df(red_flag_df, red_flag_code):
    red_flag_df['Red Flag'] = np.nan
    red_flag_df['Red Flag'].fillna(red_flag_code, inplace=True)
    red_flag_df.drop(columns=['index'], inplace=True)
    return red_flag_df

In [31]:
df_ocid = pd.DataFrame() 

In [32]:
df_NF003 = NF003(tender)
df_ocid = generate_df(df_NF003['ocid'].reset_index(), 'NF003')

In [33]:
df_NF010 = NF010(tender, montos)
df_ocid = pd.concat([df_ocid, generate_df(df_NF010['ocid'].reset_index(), 'NF010')])

In [34]:
df_NF016 = NF016()
df_ocid = pd.concat([df_ocid, generate_df(df_NF016['ocid'].reset_index(), 'NF016')])

In [35]:
df_NF018 = NF018()
df_ocid = pd.concat([df_ocid, generate_df(df_NF018['ocid'].reset_index(), 'NF018')])

In [36]:
df_NF026 = NF026()[1].drop_duplicates(subset=['ocid'])
df_ocid = pd.concat([df_ocid, generate_df(df_NF026['ocid'].reset_index(), 'NF026')])

In [37]:
df_NF027 = NF027()
df_ocid = pd.concat([df_ocid, generate_df(df_NF027['ocid'].reset_index(), 'NF027')])

In [38]:
df_NF031 = NF031()
df_ocid = pd.concat([df_ocid, generate_df(df_NF031['ocid'].reset_index(), 'NF031')])

In [39]:
df_NF042 = NF042(parties)
df_ocid = pd.concat([df_ocid, generate_df(df_NF042['ocid'].reset_index(), 'NF042')])

c:\users\sof-d\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [40]:
df_NF043 = NF043(parties)
df_ocid = pd.concat([df_ocid, generate_df(df_NF043['ocid'].reset_index(), 'NF043')])

In [41]:
df_NF044 = NF044(parties)
df_ocid = pd.concat([df_ocid, generate_df(df_NF044['ocid'].reset_index(), 'NF044')])

In [42]:
df_NF047 = NF047(parties)
df_ocid = pd.concat([df_ocid, generate_df(df_NF047['ocid'].reset_index(), 'NF047')])

In [43]:
df_NF059 = NF059()
df_ocid = pd.concat([df_ocid, generate_df(df_NF059['ocid'].reset_index(), 'NF059')])

In [44]:
df_NF061 = NF061()
df_ocid = pd.concat([df_ocid, generate_df(df_NF061['ocid'].reset_index(), 'NF061')])

In [45]:
df_NF062 = NF062()
df_ocid = pd.concat([df_ocid, generate_df(df_NF062['ocid'].reset_index(), 'NF062')])

In [46]:
df_NF063 = NF063()
df_ocid = pd.concat([df_ocid, generate_df(df_NF063['ocid'].reset_index(), 'NF063')])

In [47]:
df_ocid.reset_index(drop=True, inplace=True)

In [48]:
df_NF011 = NF011()
df_NF011['Red Flag'] = 'NF011'
df_NF011 = df_NF011.iloc[:,::-1]
df_NF011

,Red Flag,procuringEntity_name
0,NF011,BANCO CENTRAL DEL ECUADOR
1,NF011,CONSEJO DE GOBIERNO DEL REGIMEN ESPECIAL DE GA...
2,NF011,CORPORACION ELECTRICA DEL ECUADOR CELEC EP
3,NF011,CORPORACION ELECTRICA DEL ECUADOR CELEC EP.
4,NF011,CUERPO DE INGENIEROS DEL EJERCITO
5,NF011,DIRECCION DISTRITAL 02D01-GUARANDA-BOLIVAR-MTOP
6,NF011,DIRECCION DISTRITAL 13D01 - PORTOVIEJO - MANAB...
7,NF011,DIRECCION DISTRITAL 21D02C01-NUEVA LOJA-SUCUMB...
8,NF011,DIRECCION NACIONAL DE ANTINARCOTICOS
9,NF011,EMPRESA ELECTRICA REGIONAL CENTROSUR C.A.


In [49]:
df_NF019 = NF019()
df_NF019['Red Flag'] = 'NF019'
df_NF019 = df_NF019.iloc[:,::-1]
df_NF019

,Red Flag,procuringEntity_name
0,NF019,MINISTERIO DE TURISMO
1,NF019,GOBIERNO AUTONOMO DESCENTRALIZADO MUNICIPAL DE...
2,NF019,GOBIERNO PROVINCIAL DE LOS RIOS
3,NF019,CUERPO DE BOMBEROS DEL CANTÓN OTAVALO
4,NF019,DIRECCION DISTRITAL 13D10-JAMA-PEDERNALES-EDUC...
...,...,...
109,NF019,UNIDAD DE ANÁLISIS FINANCIERO Y ECONÓMICO UAFE
110,NF019,MINISTERIO DE GOBIERNO
111,NF019,MUNICIPIO DE BALAO
112,NF019,MUNICIPIO DE MEJIA


In [50]:
df_ocid = df_ocid.merge(tender[['ocid', 'procuringEntity_name']], on='ocid')
df_ocid.drop(columns=['ocid'], inplace=True)

In [51]:
df_ocid = pd.concat([df_ocid, df_NF011,df_NF019])
df_ocid

,Red Flag,procuringEntity_name
0,NF003,Empresa Pública de Áridos y Asfaltos del Azuay
1,NF018,Empresa Pública de Áridos y Asfaltos del Azuay
2,NF047,Empresa Pública de Áridos y Asfaltos del Azuay
3,NF062,Empresa Pública de Áridos y Asfaltos del Azuay
4,NF003,GOBIERNO AUTONOMO DESCENTRALIZADO MUNICIPAL DE...
...,...,...
109,NF019,UNIDAD DE ANÁLISIS FINANCIERO Y ECONÓMICO UAFE
110,NF019,MINISTERIO DE GOBIERNO
111,NF019,MUNICIPIO DE BALAO
112,NF019,MUNICIPIO DE MEJIA


In [52]:
dummies = pd.get_dummies(df_ocid['Red Flag'])
df_ocid[dummies.columns] = dummies

In [53]:
red_flags_names = df_ocid.iloc[:,2:].columns.tolist()

In [54]:
red_flags_names

['NF003',
 'NF010',
 'NF011',
 'NF016',
 'NF018',
 'NF019',
 'NF026',
 'NF027',
 'NF031',
 'NF042',
 'NF044',
 'NF047',
 'NF059',
 'NF061',
 'NF062',
 'NF063']

In [55]:
df_ocid = df_ocid.groupby('procuringEntity_name')[red_flags_names].sum().reset_index()
df_ocid

,procuringEntity_name,NF003,NF010,NF011,NF016,NF018,NF019,NF026,NF027,NF031,NF042,NF044,NF047,NF059,NF061,NF062,NF063
0,ACCION SOCIAL COMO ENTIDAD DE INCLUSION SOCIAL...,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,ACERIAS NACIONALES DEL ECUADOR SOCIEDAD ANONIM...,2,0,0,0,0,1,1,1,0,2,0,3,1,1,2,0
2,AGENCIA DE REGULACIÓN Y CONTROL FITO Y ZOOSANI...,0,0,0,0,0,0,0,0,0,24,0,49,0,0,0,0
3,"AGENCIA NACIONAL DE REGULACION, CONTROL Y VIGI...",0,0,0,0,0,0,0,0,0,3,0,6,0,0,0,0
4,AGENCIA NACIONAL DE REGULACIÓN Y CONTROL DEL T...,2,0,0,0,0,1,1,0,0,1,0,2,0,0,2,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1366,junta parroquial de cuyuja,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1367,junta parroquial la tola,0,0,0,0,0,0,0,0,0,0,0,3,0,0,0,0
1368,juntaparroquialbuenavista,0,0,0,0,0,0,0,0,0,1,0,3,0,0,0,0
1369,juntaparroquialsalango,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


#DataFrame without duplicate ocids. The clustering is going to be made based on it.
red_flags = pd.DataFrame(df_ocid['ocid'])
red_flags_names = df.iloc[:,1:-4].columns.tolist()
red_flags = df.groupby('procuringEntity_name')[red_flags_names].sum().reset_index()

red_flags.drop_duplicates(inplace=True)
red_flags = red_flags.reset_index(drop=True)

for code in dummies.columns:
    df_temp = df_ocid.groupby('ocid')[code].sum().reset_index() 
    red_flags = red_flags.merge(df_temp, on='ocid')

red_flags = red_flags.merge(tender[['ocid', 'procuringEntity_name']], on='ocid')
red_flags

In [56]:
df_ocid.to_csv('../Data_output/red_flags.csv')

## Preguntas para análisis

#### Pregunta: Para 2019 pre-pandemia, ¿qué meses del año presentaron más días de retraso en la firma de contratos de licitación?

In [ ]:
df_NF061 = NF061()

In [ ]:
df_NF061.drop(['contracts_dateSigned', 'date'], axis=1, inplace=True)

In [ ]:
df_NF061 = df_NF061.merge(releases[['ocid', 'date']], on ='ocid')

In [ ]:
df_NF061['date'] = pd.to_datetime(df_NF061['date']).dt.date
df_NF061['date'] = pd.to_datetime(df_NF061['date'])

In [ ]:
df_NF061 = df_NF061.sort_values(by='date')

In [ ]:
dfg = df_NF061.groupby(pd.Grouper(key='date', freq='M'))['contract_period'].mean().reset_index()

In [ ]:
dfg['month'] = dfg['date'].dt.month_name()

In [ ]:
dfg = dfg[dfg['contract_period'].notnull()]

In [ ]:
dfg

In [ ]:
sns.set_theme(style="darkgrid")
fig,ax = plt.subplots(figsize=(8,8))
sns.lineplot(data=dfg, x = "month", y = "contract_period",ax = ax, marker = "v", color = "b",linestyle="-.")
_=plt.xticks(rotation=90)
ax.set_xlabel("Meses")
ax.set_ylabel("Días de retraso en contratos")

#### ¿Que licitadores presentan mayor retraso en la firma de licitaciones?

In [ ]:
df_NF061 = df_NF061.merge(tender[['ocid', 'procuringEntity_name']], on='ocid')

In [ ]:
dfg2 = df_NF061.groupby('procuringEntity_name')['contract_period'].mean().reset_index()

In [ ]:
dfg2.sort_values(by='contract_period', ascending = False, inplace = True)

In [ ]:
dfg2

In [ ]:
sns.set_theme(style="darkgrid")
fig,ax = plt.subplots(figsize=(10,8))
sns.barplot(data=dfg2, y = "procuringEntity_name", x = "contract_period",ax = ax)
_=plt.xticks(rotation=90)
ax.set_ylabel("Entidad")
ax.set_xlabel("Promedio de días de retraso")